In [ ]:
########### Validation Code Pipeline
# This script is used to perform a CURIAL validation on a given clinical dataset by producing model predictions on a dataset, and comparing against a ground truth.
# By using the LFD flag, can assess on a subset of patients with LFD results
# The Admissions flag allows subsetting admitted patients from all-attenders (where available)
# The LFDComposite flag considers either a positive LFD or a positive CURIAL result as a suspected-COVID-19 case
# The LFDComposite flag requires the LFD flag to be be set to true as must subset patients to cohort who received an LFD test

# Feature sets correspond to CURIAL version (OLO+Vitals (-Rapide), Bloods+Vitals (-Lab), Bloods+Gas+Vital (-1.0, as per derivation paper))

# AS version 0.1 21 Apr 2021

In [ ]:
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from scipy.special import ndtri
from math import sqrt
from sklearn.metrics import roc_curve
from matplotlib import pyplot
import matplotlib as plt

In [ ]:
#Load dataset being evaluated (OHU / UHB / PUH / BH)
dataset = pd.read_csv('OUH Wave 2 Attendances with LFTs.csv', parse_dates=True)

In [ ]:
#Define the model being validated, the type of validation, and the settings of the model being validated

#Configuration and variables for model prediction script
# (CURIAL-Rapide, CURIAL-Lab & CURIAL-1.0 respectively) #
featureSets = ['OLO & Vitals', 'Blood & Vitals', 'Blood & Blood_Gas & Vitals'];

#Define which CURIAL sensitivies are being assessed
recalls = ['0.800','0.900']

#Define imputation method is being used during the evaluation (mode, median, mean)
imputationMethods = ['median']

#Define whether evaluating model for all-comers to hospital (ED, acute medicine, etc) or just patients who went on to be admitted
Admission = True

#Define if performing analysis just on patients with LFDs (allows comparison against LFDs)
LFD = False

#Composite pathway: if wanting to assess the combined pathway of calling COVID-19-suspected if either LFDs + CURIAL results positive, set flag to True. 
#For a standard validation, set to false
CURIALLFDComposite = False

In [ ]:
#Set date range (e.g. for OUH, during second wave from 1st Oct 2020 onwards)
dataset = dataset[dataset.ArrivalDateTime >= '2020-10-01']

#Print out size
dataset.shape

In [ ]:
#Exclude patients not tested for COVID-19
dataset = dataset[~dataset['Covid-19 Positive'].isna()]

dataset.shape

In [ ]:
#Covid test counts
dataset['Covid-19 Positive'].value_counts()

In [ ]:
# Function to calculate 95% CIs with Wilson's Method
# Source: https://gist.github.com/maidens/29939b3383a5e57935491303cf0d8e0b
# #    
#     References
#     ----------
#     [1] R. G. Newcombe and D. G. Altman, Proportions and their differences, in Statisics
#     with Confidence: Confidence intervals and statisctical guidelines, 2nd Ed., D. G. Altman, 
#     D. Machin, T. N. Bryant and M. J. Gardner (Eds.), pp. 45-57, BMJ Books, 2000. 
# #
def _proportion_confidence_interval(r, n, z):
    """Compute confidence interval for a proportion.
    
    Follows notation described on pages 46--47 of [1]. 
    
    References
    ----------
    [1] R. G. Newcombe and D. G. Altman, Proportions and their differences, in Statisics
    with Confidence: Confidence intervals and statisctical guidelines, 2nd Ed., D. G. Altman, 
    D. Machin, T. N. Bryant and M. J. Gardner (Eds.), pp. 45-57, BMJ Books, 2000. 
    """
    
    A = 2*r + z**2
    B = z*sqrt(z**2 + 4*r*(1 - r/n))
    C = 2*(n + z**2)
    return ((A-B)/C, (A+B)/C)

def sensitivity_and_specificity_with_confidence_intervals(TP, FP, FN, TN, alpha=0.95):
    """Compute confidence intervals for sensitivity and specificity using Wilson's method. 
    
    This method does not rely on a normal approximation and results in accurate 
    confidence intervals even for small sample sizes.
    
    Parameters
    ----------
    TP : int
        Number of true positives
    FP : int 
        Number of false positives
    FN : int
        Number of false negatives
    TN : int
        Number of true negatives
    alpha : float, optional
        Desired confidence. Defaults to 0.95, which yields a 95% confidence interval. 
    
    Returns
    -------
    sensitivity_point_estimate : float
        Numerical estimate of the test sensitivity
    specificity_point_estimate : float
        Numerical estimate of the test specificity
    sensitivity_confidence_interval : Tuple (float, float)
        Lower and upper bounds on the alpha confidence interval for sensitivity
    specificity_confidence_interval
        Lower and upper bounds on the alpha confidence interval for specificity 
        
    References
    ----------
    [1] R. G. Newcombe and D. G. Altman, Proportions and their differences, in Statisics
    with Confidence: Confidence intervals and statisctical guidelines, 2nd Ed., D. G. Altman, 
    D. Machin, T. N. Bryant and M. J. Gardner (Eds.), pp. 45-57, BMJ Books, 2000. 
    [2] E. B. Wilson, Probable inference, the law of succession, and statistical inference,
    J Am Stat Assoc 22:209-12, 1927. 
    """
    
    # 
    z = -ndtri((1.0-alpha)/2)
    
    # Compute sensitivity using method described in [1]
    sensitivity_point_estimate = TP/(TP + FN)
    sensitivity_confidence_interval = _proportion_confidence_interval(TP, TP + FN, z)
    
    # Compute specificity using method described in [1]
    specificity_point_estimate = TN/(TN + FP)
    specificity_confidence_interval = _proportion_confidence_interval(TN, TN + FP, z)
    
    return sensitivity_point_estimate, specificity_point_estimate, sensitivity_confidence_interval, specificity_confidence_interval

#### Function to calculate AUROC with 95% CIs
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import numpy as np
import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

In [ ]:
alpha = 0.95

def calculateMetricsWithProbs (preds, probs, ground_truth, alpha = alpha):
        precision = precision_score(ground_truth,preds,zero_division=0)
        recallAchieved = recall_score(ground_truth,preds,zero_division=0)
        accuracy = accuracy_score(ground_truth,preds)
        #auc = roc_auc_score(ground_truth,pred_probs) 
        #ns_fpr, ns_tpr, _ = roc_curve(ground_truth, pred_probs)
        tn, fp, fn, tp = confusion_matrix(ground_truth,preds).ravel()
        specificity = tn/(tn+fp)
        npv = tn/(fn+tn)
        
        #Work out AUROC using delong method (in function above)
        auc, auc_cov = delong_roc_variance(ground_truth,probs)
        auc_std = np.sqrt(auc_cov)
        lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
        auc_ci = np.around(stats.norm.ppf(lower_upper_q,loc=auc,scale=auc_std), decimals=3)
        
        #CIs for accuracy, PPV and NPV, Se, Sp
        alpha = 0.95
        z = -ndtri((1.0-alpha)/2)
        accuracy_ci = np.around(np.around(_proportion_confidence_interval(tp+tn, tp+tn+fp+fn, z), decimals=3)*100,decimals=1)
        ppv_ci = np.around(np.around(_proportion_confidence_interval(tp, tp+fp, z), decimals=3)*100,decimals=1)
        npv_ci = np.around(np.around(_proportion_confidence_interval(tn, tn+fn, z), decimals=3)*100,decimals=1)
        se_ci = np.around(np.around(_proportion_confidence_interval(tp, tp+fn, z), decimals=3)*100,decimals=1)
        sp_ci = np.around(np.around(_proportion_confidence_interval(tn, tn+fp, z), decimals=3)*100,decimals=1)
        
        #Generate strings
        accuracy_withCI = "%s%% (%s - %s)" % (np.round(accuracy*100, 1), accuracy_ci[0], accuracy_ci[1])
        ppv_withCI =  "%s%% (%s - %s)" % (np.round(precision*100, 1), ppv_ci[0], ppv_ci[1])
        npv_withCI =  "%s%% (%s - %s)" % (np.round(npv*100, 1), npv_ci[0], npv_ci[1])
        se_withCI =  "%s%% (%s - %s)" % (np.round(recallAchieved*100, 1), se_ci[0], se_ci[1])
        sp_withCI =  "%s%% (%s - %s)" % (np.round(specificity*100, 1), sp_ci[0], sp_ci[1])
        roc_withCI =  "%s (%s - %s)" % (np.round(auc, 3), auc_ci[0], auc_ci[1])
        
        #Generate ROC curves
        fpr, tpr, thresholds = roc_curve(ground_truth, probs)
        
        f1score = 2*(precision*recallAchieved)/(precision+recallAchieved)
        
        return se_withCI,sp_withCI,accuracy_withCI,roc_withCI,ppv_withCI,npv_withCI,fn,fp,f1score, fpr, tpr

def calculateMetricsWithoutProbs (preds, ground_truth, alpha = alpha):
    precision = precision_score(ground_truth,preds,zero_division=0)
    recallAchieved = recall_score(ground_truth,preds,zero_division=0)
    accuracy = accuracy_score(ground_truth,preds)
    #auc = roc_auc_score(ground_truth,pred_probs) 
    #ns_fpr, ns_tpr, _ = roc_curve(ground_truth, pred_probs)
    tn, fp, fn, tp = confusion_matrix(ground_truth,preds).ravel()
    specificity = tn/(tn+fp)
    npv = tn/(fn+tn)
    f1score = 2*(precision*recallAchieved)/(precision+recallAchieved)

    #CIs for accuracy, PPV and NPV, Se, Sp
    alpha = 0.95
    z = -ndtri((1.0-alpha)/2)
    accuracy_ci = np.around(np.around(_proportion_confidence_interval(tp+tn, tp+tn+fp+fn, z), decimals=3)*100,decimals=1)
    ppv_ci = np.around(np.around(_proportion_confidence_interval(tp, tp+fp, z), decimals=3)*100,decimals=1)
    npv_ci = np.around(np.around(_proportion_confidence_interval(tn, tn+fn, z), decimals=3)*100,decimals=1)
    se_ci = np.around(np.around(_proportion_confidence_interval(tp, tp+fn, z), decimals=3)*100,decimals=1)
    sp_ci = np.around(np.around(_proportion_confidence_interval(tn, tn+fp, z), decimals=3)*100,decimals=1)
    
    #Generate strings
    accuracy_withCI = "%s%% (%s - %s)" % (np.round(accuracy*100, 1), accuracy_ci[0], accuracy_ci[1])
    ppv_withCI =  "%s%% (%s - %s)" % (np.round(precision*100, 1), ppv_ci[0], ppv_ci[1])
    npv_withCI =  "%s%% (%s - %s)" % (np.round(npv*100, 1), npv_ci[0], npv_ci[1])
    se_withCI =  "%s%% (%s - %s)" % (np.round(recallAchieved*100, 1), se_ci[0], se_ci[1])
    sp_withCI =  "%s%% (%s - %s)" % (np.round(specificity*100, 1), sp_ci[0], sp_ci[1])
    
    return se_withCI,sp_withCI,accuracy_withCI,np.NaN,ppv_withCI,npv_withCI,fn,fp,f1score


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov

In [ ]:
#Create Results output dataframe
results = pd.DataFrame(columns=['Feature Set','CURIAL Targetted Recall','Imputation','CURIAL Recall during training CV','Achieved Sensitivity','Specificity','Accuracy','AUROC','Precision','NPV','n False Negative','n False Positive','F1'])
i=1;

ax=pyplot.axes()

for featureSet in featureSets:
    print ("***Feature Set: "+featureSet)
        #results= {}
    
    for recall in recalls:
        for imputationMethod in imputationMethods:
            #Scalar File
            scalarFile = '../norm_'+featureSet+'.pkl'
            modelFile = '../Results/'+featureSet+'/Recall_'+recall+'/fitted_models_dict.pkl'
            imputerFile =  '../imputer_'+featureSet+'_'+imputationMethod+'.pkl'
            featureListFile = '../featurelist_'+featureSet+'.pkl'
            
            #Restricting to past Sept 2020
            dataset=dataset[dataset.ArrivalDateTime > '2020-09-01' ]
            
            if Admission:
                dataset = dataset[dataset.Admission == 1]
            
            #dataset=dataset[dataset.ArrivalDateTime > '2020-12-22' ]

            #Import scalar
            scaler = pd.read_pickle(scalarFile)

            #Import imputer
            imputer = pd.read_pickle(imputerFile)

            #Import models
            models_dict = pd.read_pickle(modelFile)

            #Import feature list
            featureList = pd.read_pickle(featureListFile)
            #print (featureList)

            #Import performance file
            performanceFile = pd.read_csv('../Results/'+featureSet+'/Recall_'+recall+'/10CV_Mean_Multiple_Imputation.csv')
            thresholdForSensitivity = performanceFile['Threshold'][0]
            
            if LFD == True:
            #Filter out only presentations with LFDs if LFD is true
                presentationsWithLFDs = dataset[(dataset.Lateral_flow_result=='Positive') | (dataset.Lateral_flow_result=='Negative')]
            else:
                presentationsWithLFDs = dataset

            #Select out features, in the model's required order
            predictionSet = presentationsWithLFDs[featureList]

            #Impute missing data using the correct imputer
            imputedPredictionSet = imputer.transform(predictionSet.values)

            #Apply scalar transform
            scaledPredictionSet = scaler.transform(imputedPredictionSet)

            #Select key for model with the correct Feature Set and with each Imputation
            selectedModel = "['covid_vs_all', True, 20, '"+imputationMethod+"', '"+featureSet+"', 'XGB', False, None]"

            #Make predictions
            pred_probs = models_dict[selectedModel].predict_proba(pd.DataFrame(scaledPredictionSet))[:,1]
            preds = np.where(pred_probs>thresholdForSensitivity,1,0)

            #Configure Output Df
            outputDf = pd.DataFrame(presentationsWithLFDs[['ClusterID','EpisodeID','ArrivalDateTime','Covid-19 Positive','Lateral_flow_result']])
            lfd_ids = {'Negative': 0, 'Positive': 1}

            def featurize_protected(df, protected, id_map):
                col = df[protected].copy()
                col.replace(id_map, inplace=True)
                return col
            
            lfd_col = featurize_protected(outputDf, 'Lateral_flow_result', lfd_ids)
            outputDf = outputDf.fillna({'Covid-19 Positive': 0})
            outputDf['LFD_binary_result'] = lfd_col

            #Add in Model Predictions
            outputDf['CURIAL_Preds'] = pred_probs

            #Apply Threshold
            outputDf['CURIAL_Output'] = preds
            
            if CURIALLFDComposite:
                #Generate LFD-Curial Composite Score
                outputDf['LFD-Curial-Composite'] = (outputDf['CURIAL_Output'] + outputDf['LFD_binary_result']).clip(upper=1)
                outputDf['LFD-Curial-Composite_prob'] = (outputDf['CURIAL_Preds'] + outputDf['LFD_binary_result']).clip(upper=1)
            
    
            #Print out number positive and number negative
            #outputDf['CURIAL_Output'].value_counts()

            # Sensitivity during CV Val
            crossValidationSensitivity = performanceFile['Recall'][0]

            #Calculate summary metrics for CURIAL Model
            recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score, fpr, tpr = calculateMetricsWithProbs (preds, pred_probs, outputDf['Covid-19 Positive'])
            #Write results
            metrics = [featureSet,recall,imputationMethod,crossValidationSensitivity,recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score]
            results.loc[i,:] = metrics
            i=i+1

            #------------
            #Now calculate summary metrics for CURIAL-LFD Composite
            if CURIALLFDComposite:
                #recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score = calculateMetricsWithoutProbs (outputDf['LFD-Curial-Composite'], outputDf['Covid-19 Positive'])
                recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score, fpr, tpr = calculateMetricsWithProbs (outputDf['LFD-Curial-Composite'], outputDf['LFD-Curial-Composite_prob'], outputDf['Covid-19 Positive'])
                metrics = ['Composite LFD-CURIAL: '+featureSet,recall, imputationMethod,np.NaN,recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score]
                results.loc[i,:] = metrics
                i=i+1
                                
            if CURIALLFDComposite:
                ax.plot(fpr, tpr, marker='.', label=featureSet)
            else:
                ax.plot(fpr, tpr, marker='.', label='Composite LFD-CURIAL: '+featureSet)

            #print (featureSet+" performance at Se: "+recall+' VS LFD as Gold Standard')
            #print(classification_report(outputDf['LFD_binary_result'], outputDf['CURIAL_Output']))

if LFD:
    print ("**LFD Performance")
    print(classification_report(outputDf['Covid-19 Positive'], outputDf['LFD_binary_result']))

In [ ]:
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:
#Plot a calibration curve
from sklearn.calibration import calibration_curve
from sklearn.metrics import roc_curve
from matplotlib import pyplot

outputDf=outputDf

# reliability diagram
fop, mpv = calibration_curve(outputDf['Covid-19 Positive'], outputDf['CURIAL_Preds'], n_bins=15, normalize=True)
# plot perfectly calibrated
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
pyplot.plot(mpv, fop, marker='.')
pyplot.show()

In [ ]:
#Add raw LFD performance on this cohort
if LFD:
    recallAchieved,specificity,accuracy,auc,precision,npv,fn,fp,f1score = calculateMetricsWithoutProbs (outputDf['LFD_binary_result'], outputDf['Covid-19 Positive'])
    metrics = ['Lateral Flow Tests',np.NaN,np.NaN,np.NaN,recallAchieved,specificity,accuracy,np.NaN,precision,npv,fn,fp,f1score]
    results.loc[0,:] = metrics

In [ ]:
# Print out evaluation results
results.drop(columns=['Imputation','CURIAL Recall during training CV'], inplace=True)
results